In [1]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [3]:
model = torch.load('./model/D_B.pt').eval()

In [17]:
d([[14649,   637,    35,  3163,   528,   266,  6466,     3,  1878,  1261,
           548, 25556,    29,     5,  3204, 11141,  1392,  7667,     3,    35,
            74, 25556,     5,     3,   454,     1,     3,  2820,   272,     3,
             3,     3,   637,   781, 31660,    61, 23986,  1185, 23986, 19876,
         32075,   644,  2263,     3,  2739,  2995,  7672,    74,   219,  4042,
         12424,  1982,     7,     3,   505,     1,     0, 11093,     3,     0,
             3,   316,     1, 16284, 19358,  1811,     0,     3,     3,     3,
             3,   510,     3, 17836,  6101,  1122, 31848,     3, 18122, 19709,
         13411,     1, 23986, 22907,     0,     1,     5,  3067, 11591,     3,
          1983,     3,   644,     3,    86, 25533,     3,     1,  1624,  8330,
            86,     3,     3,   644,  2167, 24566,  9654,   878, 31391,  3360,
         25385, 30615,   117,     1],
        [ 1203,    67,  1352, 14668,     6,     3,    26,  7118,   558,    92,
           157,     7,   548,  7716, 14002,   745,   183,  2889,  3296,  7118,
            74,  4276,    15,     7,   403,     3, 16067,   745,     5,  1301,
            26,  7118,   558,     3,   362,     7,  4270,  5240,     7,   745,
             6,  6034,  3296,  7118,  5854,  6248,    15,     7,   319,  4909,
         30249,   745,     6,  1149,    26,  7118,     3, 17199,    89,    26,
             6,   181, 27551,  8827,     6,    78,    26,  7118,   558,   584,
             9,     7,  1806,  2619,  6975,   745,     6,   211, 13607,  7118,
           558,     3,    15,     7,     3,     3,  4236,   745,     6,   292,
          3296,  7118,   558,     3, 29277,     7,   183, 11994,   745,     6,
             3,    26,  7118,   558,  9924,   295,     7, 19815,   319,   122,
           745,     6,     3,     1]])

['Natürlich dieseen wurden je eine solche dieser Art vergetroffenn. Hier handelt Ausebenfalls en dergetroffen.  H  Eine B    diese Ver Genehmigung) Nutzen Wir Nutzen Maßnahmen Das Hand  Wie gegen Frau der auf dazu ebensomals  8 Mehr   Die ähnlich Gebiet Zu     über  Bristol darauf Es Mängel  Staat tätig erklärt Nutzen kostet. weiter Pie  Act  Das  In gefällt  Red klar In   Das Diese behandelt Dies Da Konsequenz bereits Ausnahme Afrika;',
 'Also die danngesagt, daß wir alsoks ver richtiggemäß haben am damitsondernaß der keines aus holen haben. wenndaß wir ichs heute tuns haben, geradesondernaß neu Fres auch einmalKampagne haben, unsdaß  demanderfd, mitgeschafft hatten, sodaß wir Vas gut selbstgegeben haben, dasanimaß wir es  bietet haben, Siesondernaß wir vermutlichs amwagen haben, daß wir darüberents teilweise auchg haben, ']

In [9]:

label = en(['Mr President, Commissioner, I would like to thank Mrs Schroedter for an excellent report.', 'She has gone into the issue in some depth and in the committee debate she took account of many of the amendments that have been tabled regarding this report.'])
pred = en(['Mr President, Commissioner President-in-Office of the Council, Commissioner Commissioner Commissioner, Commissioner Commissioner Commissioner Commissioner, Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner, ladies and gentlemen, Commissioner Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner, Commissioner Commissioner,', 'I also welcome the comments made on behalf of the Committee on Legal Affairs and the Internal Market and the Committee on Economic and Monetary Affairs.'])

In [6]:
label = torch.Tensor([[ 2167,  1392, 19107,    64,    67,     3,     2, 20181,    74,  2995,
          7497,    52,  2880, 13896,   537,     3, 28180, 19333,  1149,    67,
           493,  7733, 15635,   425,  9687,    49,   961,  6186, 14707,     6,
            67, 26740,    74, 27579,  1324,     7,  5578,     7,   436,     5,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [   86, 20878, 10993,  7852,     3,    15,     7,    16,   177,     3,
          5802,  3093, 12834,   468,    74,    86, 18992,  8369,    74, 25886,
         14080,     7,     6,     3, 21984,    23,    67,  6822, 12880,    16,
           645, 13010,   115,    40,  5588,   324, 15833,   157,  6850,    15,
             6,    16,    73,   172,    83,  7423,  2978,  4713,  1885, 19580,
             3,  9096,     6,     3,    26,  7118,    67, 18209,    18,    64,
         13848,  1639,  5348,    15,   219,     3, 20112,    49, 21621,   170,
          4499, 25477,  3163,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]])

In [10]:
#real sentences
x = label[0].cuda()
#delte 0 give huge diff
       
# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_label =  model.classifier(distr)#(bs,1)
print(ret_label)

print(model(x,x_attn))

tensor([[ 1363,  1661,     6, 14595,     6,    27,   133,   114,    12,  2763,
          8667,   180, 10363,    15,    26,   449,    21,    46,  1287,   934,
             5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  451,    65,  2767,   139,     8,   962,    16,   128,  4963,    11,
            16,     8,  4492,  5054,   255,   808,   905,    13,   186,    13,
             8, 12123,     7,    24,    43,   118,   953,    26,  1918,    48,
           934,     5,     1]], device='cuda:0')
torch.Size([2, 33])
		---------embedding
torch.Size([2, 33, 512])
tensor([[[-15.8327,   0.3196,   3.1946,  ...,  -4.6197,  27.3722,  17.6071],
         [ -1.9148,  -6.4818,  28.3587,  ..., -56.7450, -71.9382,   8.4272],
         [ 12.5716,   7.8596, -10.9473,  ...,   7.8750,  -6.7155,  -3.3432],
         ...,
         [ -1.9089,   0.1207,  -7.0306,  ...,  -0.4225,   2.6453,  -2.8076],
         [ -1.9089,   0.1207,  -7.0306,  ...,  -0.4225,   

In [11]:
#real sentences
x = pred[0].cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_pred =  model.classifier(distr)#(bs,1)
print(ret_pred)

torch.Size([2, 256])
		---------embedding
torch.Size([2, 256, 512])
tensor([[[-15.8327,   0.3196,   3.1946,  ...,  -4.6197,  27.3722,  17.6071],
         [ -1.9148,  -6.4818,  28.3587,  ..., -56.7450, -71.9382,   8.4272],
         [ 12.5716,   7.8596, -10.9473,  ...,   7.8750,  -6.7155,  -3.3432],
         ...,
         [-27.5763, -20.2604,   2.1405,  ...,  -6.3479, -36.1910,  16.2589],
         [ 12.5716,   7.8596, -10.9473,  ...,   7.8750,  -6.7155,  -3.3432],
         [ 12.5838,   8.1357, -11.4688,  ...,   7.7555,  -7.2309,   0.8988]],

        [[ 15.7629,   3.0036,  14.8671,  ..., -11.0151,  -4.3929,   5.6647],
         [ 12.8145,   4.5959,  -1.3623,  ...,  -0.2104, -33.6627,  34.2502],
         [ 24.0963,  -6.9128, -34.4636,  ..., -18.3673,  21.6795,   5.4308],
         ...,
         [ -1.9089,   0.1207,  -7.0306,  ...,  -0.4225,   2.6453,  -2.8076],
         [ -1.9089,   0.1207,  -7.0306,  ...,  -0.4225,   2.6453,  -2.8076],
         [ -1.9089,   0.1207,  -7.0306,  ...,  -0.4225,

In [12]:
criterionGAN = torch.nn.MSELoss()
pred_real = ret_label
print(pred_real)
loss_D_real = criterionGAN(pred_real, torch.ones((pred_real.shape[0],1),device='cuda'))
print(loss_D_real)
# Fake
pred_fake = ret_pred
loss_D_fake = criterionGAN(pred_fake, torch.zeros((pred_fake.shape[0],1),device='cuda'))
# Combined loss and calculate gradients
print(loss_D_fake)


loss_D = (loss_D_real + loss_D_fake) * 0.5
print(loss_D)

tensor([[1.0234],
        [0.9825]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(0.0004, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5338, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2671, device='cuda:0', grad_fn=<MulBackward0>)


,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier

## FAKE dont end with 1, cuz i use model.generate but not self.generate